# CONFLUENCE Tutorial - 6: Regional Domain Modeling (Iceland)

## Introduction
This tutorial marks a fundamental shift in scale and approach from single-watershed modeling to regional domain modeling. Instead of focusing on one watershed draining to a single outlet, we now model an entire region containing multiple independent drainage systems. Using Iceland as our example, we demonstrate how CONFLUENCE handles complex regional hydrology including coastal watersheds, multiple drainage basins, and diverse hydrological regimes within a single integrated modeling framework.

## Regional Domain Modeling Philosophy
Regional domain modeling treats an entire geographic region such as a country, province, or large administrative area as the modeling domain, fundamentally departing from traditional watershed-centric approaches. This methodology encompasses multiple independent watersheds within the domain, including coastal watersheds that drain directly to ocean boundaries rather than to interior confluence points. The approach enables comprehensive regional water resources assessment by providing understanding of water availability patterns across entire geographic regions while simultaneously addressing diverse hydrological conditions including multiple climate zones, varied topographic settings, and contrasting hydrological regimes within a unified analytical framework.

The conceptual differences between watershed and regional modeling are substantial and fundamental. Drainage patterns shift from single outlet points to multiple independent outlets, while domain boundaries transition from topographic divides to administrative or geographic boundaries. Spatial scales expand from tens or thousands of square kilometers to tens of thousands or hundreds of thousands of square kilometers. Outlet systems evolve from rivers and streams to include both riverine systems and coastal discharge points. The analytical purpose advances from basin-specific studies to comprehensive regional water resources assessment and management.

## Case Study: Iceland Regional Analysis
Iceland provides an exceptional case study for regional modeling through its unique combination of geographic characteristics, hydrological diversity, and practical advantages for comprehensive analysis. The island nation offers clear regional boundaries with diverse drainage systems spanning approximately 103,000 square kilometers, creating a manageable demonstration scale that remains large enough to illustrate regional concepts effectively. The volcanic landscape creates complex topography that generates numerous independent watersheds, while the clear geographic boundaries of the island setting eliminate the boundary condition complexities that complicate continental regional studies.

The hydrological diversity of Iceland encompasses glacial systems with ice-dominated watersheds that exhibit unique hydrological behaviors, volcanic terrain with permeable basalts that significantly affect groundwater flow patterns, coastal climate zones with maritime influences that create strong seasonal variations, and elevation gradients ranging from sea level to over 2000 meters that drive diverse hydrological responses. This diversity enables demonstration of regional modeling capabilities across fundamentally different hydrological regimes within a single coherent framework.

Practical advantages include excellent data availability with good coverage of meteorological and topographic datasets, computational feasibility that balances demonstration scale with local execution capabilities, and clear boundary definition through the island setting that eliminates complex boundary condition specifications required for continental applications.

## Technical Implementation Framework
Regional modeling requires several key technical approaches that distinguish it from traditional watershed-specific methods. Bounding box delineation defines the region using geographic coordinates rather than pour point specifications, enabling comprehensive coverage of administrative or natural regions. Coastal watershed inclusion captures watersheds that drain directly to ocean boundaries, ensuring complete regional water accounting. Multiple drainage system handling accommodates independent watersheds without requiring upstream-downstream connectivity assumptions. Regional forcing application distributes meteorological data across diverse topographic and climate conditions while maintaining spatial coherence and physical realism.

Critical configuration parameters undergo substantial modification for regional applications. Domain setup parameters change to disable pour point delineation in favor of bounding box definitions, specify regional extents rather than watershed-specific coordinates, and enable coastal watershed inclusion to capture ocean-draining basins. Computational considerations require higher stream threshold values to manage the increased complexity of regional stream networks, careful balance between spatial resolution detail and computational feasibility, and appropriate planning for extended processing times due to regional scale demands.

## Learning Objectives and Tutorial Structure
Through this tutorial, you will master the fundamental principles of scaling hydrological modeling from individual watersheds to comprehensive regions, understand the technical requirements and implementation strategies for regional domain modeling, develop capabilities for handling multiple independent drainage systems within unified frameworks, comprehend the scientific applications and policy relevance of regional-scale hydrology, and appreciate the computational and data management challenges inherent in large-scale modeling applications.

This tutorial represents the largest spatial scale in our notebook series yet, and demonstrates CONFLUENCE's capability to handle complex regional hydrology while maintaining the same workflow efficiency and scientific rigor established in smaller-scale applications. The methodological foundation established here directly enables the continental-scale applications and large-sample studies that follow in subsequent tutorials.

## Step 1: Regional Domain Setup with Multi-Watershed Configuration
This tutorial marks a shift from watershed-scale to regional-scale hydrological modeling. Moving beyond the single-watershed focus of our previous tutorials, we now model an entire region containing multiple independent drainage systems. Using Iceland as our case study, we demonstrate how CONFLUENCE scales from basin-specific modeling to comprehensive regional water resources assessment across diverse hydrological regimes.

The same CONFLUENCE framework seamlessly scales from single-watershed to regional modeling while maintaining workflow consistency and scientific rigor across this dramatic increase in spatial scope and hydrological complexity.


In [ ]:
# Import the libraries we'll need in this notebook
import sys
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from shapely.geometry import box
import contextily as cx
from datetime import datetime
import xarray as xr
import warnings

# Suppress specific warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import CONFLUENCE
from CONFLUENCE import CONFLUENCE

# Set up plotting style
plt.style.use('default')
%matplotlib inline

# =============================================================================
# CONFIGURATION FOR REGIONAL ICELAND MODELING
# =============================================================================

# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/Users/darrieythorsson/compHydro/data/CONFLUENCE_data')  # ← Update this path
#CONFLUENCE_DATA_DIR = Path('/path/to/your/CONFLUENCE_data') 

# Verify paths exist
if not CONFLUENCE_CODE_DIR.exists():
    raise FileNotFoundError(f"CONFLUENCE code directory not found: {CONFLUENCE_CODE_DIR}")

if not CONFLUENCE_DATA_DIR.exists():
    print(f"Data directory doesn't exist. Creating: {CONFLUENCE_DATA_DIR}")
    CONFLUENCE_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Load Iceland configuration from template
iceland_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_Iceland.yaml'
with open(iceland_config_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update for tutorial-specific settings
config_updates = {
    'CONFLUENCE_CODE_DIR': str(CONFLUENCE_CODE_DIR),
    'CONFLUENCE_DATA_DIR': str(CONFLUENCE_DATA_DIR),
    'DOMAIN_NAME': 'Iceland',
    'EXPERIMENT_ID': 'regional',
    'EXPERIMENT_TIME_START': '2011-01-01 01:00',
    'EXPERIMENT_TIME_END': '2013-12-31 23:00',  # Shorter period for tutorial efficiency
}

config_dict.update(config_updates)

# Save tutorial configuration
tutorial_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_iceland_tutorial.yaml'
with open(tutorial_config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)

print(f"✅ Regional Iceland configuration saved: {tutorial_config_path}")

# Initialize CONFLUENCE with tutorial config
confluence = CONFLUENCE(tutorial_config_path)

# Initialize project directory structure
project_dir = confluence.managers['project'].setup_project()

# For regional modeling, we still need a pour point file for technical reasons,
# but it won't be used for actual delineation
pour_point_path = confluence.managers['project'].create_pour_point()

print(f"   📁 Project directory: {project_dir}")


## Step 2: Regional Multi-Watershed Delineation and Coastal Integration
The transition to regional modeling requires a fundamentally different approach to domain delineation. Instead of tracing upstream from a single pour point, we now identify all independent drainage systems within a geographic region, including the critical inclusion of coastal watersheds that drain directly to ocean boundaries. This represents a paradigm shift from watershed-centric to region-centric hydrological analysis.
### Scientific Context: Regional Drainage System Analysis
Regional multi-watershed delineation operates on fundamentally different principles compared to traditional watershed-specific approaches. Bounding box definition establishes geographic region boundaries based on administrative or natural regional limits rather than topographic watershed constraints, enabling comprehensive coverage of entire administrative units or natural regions. Independent drainage systems are accommodated within a single modeling framework, allowing multiple unconnected watersheds to be analyzed simultaneously without requiring artificial connectivity assumptions. Coastal watershed integration becomes critically important as many basins drain directly to ocean boundaries rather than to inland confluence points, necessitating explicit inclusion of these ocean-draining systems that traditional watershed modeling often overlooks. Regional stream networks comprise multiple independent stream systems that lack upstream-downstream connectivity between different watershed units, requiring specialized handling of network topology and routing algorithms. Outlet multiplicity characterizes regional systems through many independent discharge points rather than the single downstream outlet typical of watershed-specific studies.

The critical differences from watershed modeling extend across multiple fundamental aspects of hydrological analysis. Domain logic transitions from topographic divides that define natural watershed boundaries to administrative or geographic boundaries that define regions of interest for water resources management. Outlet identification evolves from single known points with established gauge networks to multiple unknown coastal outlets that require systematic identification and characterization. Connectivity assumptions shift from connected drainage networks with clear upstream-downstream relationships to independent drainage systems that operate without inter-basin connectivity. Boundary conditions change from watershed edge specifications to complex ocean interface definitions that require specialized treatment of coastal discharge processes.

The regional approach captures the complete hydrological picture of a geographic area, providing essential information for comprehensive water resources assessment, climate impact studies, and regional policy applications that require understanding of hydrological processes across administrative boundaries rather than natural watershed limits.

In [ ]:
# Acquire regional geospatial data
#confluence.managers['data'].acquire_attributes()
print("✅ Regional geospatial data acquisition complete")

# Execute regional domain delineation
watershed_path = confluence.managers['domain'].define_domain()
print("✅ Regional multi-watershed delineation complete")

# Execute domain discretization for regional multi-watershed system
hru_path = confluence.managers['domain'].discretize_domain()


## REGIONAL DRAINAGE SYSTEM ANALYSIS

In [ ]:

# Load and analyze delineated watersheds
basin_path = project_dir / 'shapefiles' / 'river_basins'
network_path = project_dir / 'shapefiles' / 'river_network'

basin_count = 0
basin_files = []
basins_gdf = None

if basin_path.exists():
    basin_files = list(basin_path.glob('*.shp'))
    if basin_files:
        try:
            basins_gdf = gpd.read_file(basin_files[0])
            basins_gdf = basins_gdf.to_crs(epsg=32627)
            basin_count = len(basins_gdf)
            
            print(f"✅ Regional watersheds successfully delineated")
            print(f"   Total watersheds: {basin_count}")
            
            # Calculate regional statistics
            if not basins_gdf.empty:
                total_area = basins_gdf.geometry.area.sum() / 1e6  # Convert to km²
                avg_area = total_area / basin_count
                max_area = basins_gdf.geometry.area.max() / 1e6
                min_area = basins_gdf.geometry.area.min() / 1e6
                
                print(f"   Total regional area: {total_area:,.0f} km²")
                print(f"   Average watershed size: {avg_area:.1f} km²")
                print(f"   Watershed size range: {min_area:.1f} to {max_area:.1f} km²")
                
                # Analyze watershed characteristics if available
                if 'elevation' in basins_gdf.columns:
                    elev_range = basins_gdf['elevation'].max() - basins_gdf['elevation'].min()
                    print(f"   Elevation diversity: {basins_gdf['elevation'].min():.0f}m to {basins_gdf['elevation'].max():.0f}m ({elev_range:.0f}m span)")
                
        except Exception as e:
            print(f"❌ Error reading regional basin data: {str(e)}")
    else:
        print(f"❌ No basin shapefiles found in {basin_path}")
else:
    print(f"❌ Basin directory not found: {basin_path}")

# Analyze regional stream network
network_count = 0
network_files = []
rivers_gdf = None

if network_path.exists():
    network_files = list(network_path.glob('*.shp'))
    if network_files:
        try:
            rivers_gdf = gpd.read_file(network_files[0])
            network_count = len(rivers_gdf)
            
            print(f"✅ Regional stream network created")
            print(f"   Stream segments: {network_count}")
            
            if 'Length' in rivers_gdf.columns:
                total_length = rivers_gdf['Length'].sum() / 1000  # Convert to km
                print(f"   Total stream length: {total_length:,.0f} km")
                
        except Exception as e:
            print(f"❌ Error reading regional stream network: {str(e)}")
    else:
        print(f"❌ No stream network files found in {network_path}")
else:
    print(f"❌ Stream network directory not found: {network_path}")

# =============================================================================
# REGIONAL DOMAIN VISUALIZATION
# =============================================================================

print(f"\n🗺️  Creating regional domain visualization...")

if basins_gdf is not None and not basins_gdf.empty:
    
    fig, axes = plt.subplots(1, 2, figsize=(18, 9))
    
    # Left plot: Regional watersheds
    ax1 = axes[0]
    
    if 'GRU_ID' in basins_gdf.columns:
        basins_gdf.plot(ax=ax1, column='GRU_ID', cmap='tab20', 
                       edgecolor='black', linewidth=0.8, alpha=0.7, legend=False)
    else:
        basins_gdf.plot(ax=ax1, cmap='tab20', 
                       edgecolor='black', linewidth=0.8, alpha=0.7, legend=False)
    
    # Add stream network if available
    if rivers_gdf is not None:
        rivers_gdf.plot(ax=ax1, color='blue', linewidth=1, alpha=0.8)
    
    ax1.set_title(f'Iceland Regional Domain\n{basin_count} Independent Watersheds', 
                 fontsize=14, fontweight='bold')
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.grid(True, alpha=0.3)
    
    # Add regional statistics
    if total_area:
        ax1.text(0.02, 0.98, 
                f'Regional Coverage\n{basin_count} watersheds\n{total_area:,.0f} km²\nCoastal drainage',
                transform=ax1.transAxes, va='top',
                bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.5'),
                fontsize=11)
    
    # Right plot: Regional characteristics analysis
    ax2 = axes[1]
    
    # Create watershed size distribution if data available
    if basins_gdf is not None and not basins_gdf.empty:
        watershed_areas = basins_gdf.geometry.area / 1e6  # Convert to km²
        
        # Histogram of watershed sizes
        ax2.hist(watershed_areas, bins=min(20, basin_count//2), 
                color='skyblue', alpha=0.7, edgecolor='navy')
        ax2.set_xlabel('Watershed Area (km²)')
        ax2.set_ylabel('Number of Watersheds')
        ax2.set_title('Regional Watershed Size Distribution', fontweight='bold')
        ax2.grid(True, alpha=0.3)
        
        # Add statistics
        stats_text = (f"Total: {basin_count} watersheds\n"
                     f"Mean: {watershed_areas.mean():.1f} km²\n"
                     f"Median: {watershed_areas.median():.1f} km²\n"
                     f"Range: {watershed_areas.min():.1f}-{watershed_areas.max():.1f} km²")
        
        ax2.text(0.98, 0.98, stats_text, transform=ax2.transAxes,
                bbox=dict(facecolor='white', alpha=0.8), fontsize=10,
                ha='right', va='top')
    else:
        ax2.text(0.5, 0.5, 'Watershed analysis\nnot available', 
                transform=ax2.transAxes, ha='center', va='center',
                fontsize=16, bbox=dict(facecolor='lightgray', alpha=0.5))
        ax2.set_title('Regional Analysis', fontweight='bold')
    
    plt.suptitle(f'Regional Multi-Watershed Delineation: Iceland Domain Analysis', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print(f"✅ Regional domain visualization complete")
    
else:
    print(f"❌ Cannot create visualization - regional basin data not available")




print(f"\n📊 Regional Computational Unit Analysis...")

# Load and analyze discretized regional domain
catchment_path = project_dir / 'shapefiles' / 'catchment'
hru_count = 0
hru_gdf = None

if catchment_path.exists():
    hru_files = list(catchment_path.glob('*.shp'))
    if hru_files:
        try:
            hru_gdf = gpd.read_file(hru_files[0])
            hru_count = len(hru_gdf)
            
            print(f"✅ Regional computational units created")
            print(f"   Total HRUs/GRUs: {hru_count}")
            print(f"   Base watersheds: {basin_count}")
            
            if basin_count > 0:
                avg_units_per_watershed = hru_count / basin_count
                print(f"   Average units per watershed: {avg_units_per_watershed:.1f}")
            
            # Analyze computational complexity
            if 'GRU_ID' in hru_gdf.columns:
                unique_grus = hru_gdf['GRU_ID'].nunique()
                print(f"   Unique GRUs: {unique_grus}")
                
                # Show distribution across watersheds
                gru_counts = hru_gdf.groupby('GRU_ID').size()
                print(f"   GRU size range: {gru_counts.min()} to {gru_counts.max()} units")
                
        except Exception as e:
            print(f"❌ Error analyzing regional computational units: {str(e)}")
    else:
        print(f"📋 No discretized catchment files found yet")
else:
    print(f"📋 Catchment directory not found: {catchment_path}")


## Step 3: Regional Multi-Watershed Data Pipeline
The same model-agnostic preprocessing framework now scales to handle multiple independent drainage systems across an entire region, representing the most computationally demanding and hydrologically diverse configuration in our tutorial series. Unlike previous tutorials that processed connected watersheds or sub-watersheds, we now handle dozens of independent drainage systems with diverse hydrological regimes, coastal boundaries.

In [ ]:
# =============================================================================
# STEP 3: REGIONAL MULTI-WATERSHED DATA PIPELINE
# =============================================================================

# Check if forcing data acquisition is needed
forcing_dir = project_dir / 'forcing' / 'raw_data'
if not forcing_dir.exists() or len(list(forcing_dir.glob('*.nc'))) == 0:    
    # confluence.managers['data'].acquire_forcings()
    print("✅ Regional forcing data acquisition complete")
else:
    print(f"\n✅ Regional forcing data available")
    print(f"   Reusing meteorological data for regional domain")

#Executing Regional Model-Agnostic Preprocessing...
confluence.managers['data'].run_model_agnostic_preprocessing()
print("✅ Regional model-agnostic preprocessing complete")

#Executing Regional Model-Specific Preprocessing...
confluence.managers['model'].preprocess_models()
print("✅ Regional model-specific preprocessing complete")


## Step 4: Regional Multi-Watershed Model Execution
The same SUMMA process-based physics now executes across multiple independent drainage systems spanning an entire region. This integration of regional-scale meteorological forcing with multiple unconnected watershed systems demonstrates how the same computational framework scales to handle comprehensive regional water resources assessment.

In [ ]:
# =============================================================================
# STEP 4: REGIONAL MULTI-WATERSHED MODEL EXECUTION
# =============================================================================

# Execute the regional model system
print(f"\n⚙️  Running regional multi-watershed simulation...")
confluence.managers['model'].run_models()

print("✅ Regional multi-watershed simulation complete")


## Step 5: Regional Multi-Watershed Analysis 


In [ ]:
# =============================================================================
# STEP 5: REGIONAL MULTI-WATERSHED ANALYSIS AND COMPARATIVE ASSESSMENT
# =============================================================================

print(f"\n🌍 Loading Regional Multi-Watershed Simulation Results...")

# Load regional simulation outputs
simulation_dir = project_dir / 'simulations' / config_dict['EXPERIMENT_ID']
summa_dir = simulation_dir / 'SUMMA'
routing_dir = simulation_dir / 'mizuRoute'

# Initialize variables for regional analysis
regional_summa_data = None
regional_routing_data = None
regional_analysis_possible = False

# Load SUMMA regional outputs
summa_files = list(summa_dir.glob('*day.nc')) if summa_dir.exists() else []
if summa_files:
    try:
        regional_summa_data = xr.open_dataset(summa_files[0])
        print(f"✅ Regional SUMMA data loaded")
        print(f"   HRUs: {regional_summa_data.dims.get('hru', 'unknown')}")
        print(f"   Time steps: {regional_summa_data.dims.get('time', 'unknown')}")
        print(f"   Variables: {len(regional_summa_data.data_vars)} hydrological components")
        regional_analysis_possible = True
    except Exception as e:
        print(f"⚠️  Could not load SUMMA data: {e}")
else:
    print(f"⚠️  No SUMMA output files found")

# Load mizuRoute regional outputs
routing_files = list(routing_dir.glob('*.nc')) if routing_dir.exists() else []
if routing_files:
    try:
        regional_routing_data = xr.open_dataset(routing_files[0])
        print(f"✅ Regional mizuRoute data loaded")
        print(f"   Stream segments: {regional_routing_data.dims.get('seg', 'unknown')}")
        print(f"   Time steps: {regional_routing_data.dims.get('time', 'unknown')}")
        if 'IRFroutedRunoff' in regional_routing_data.data_vars:
            print(f"   Routed runoff: Available for regional outlet analysis")
    except Exception as e:
        print(f"⚠️  Could not load mizuRoute data: {e}")
else:
    print(f"⚠️  No mizuRoute output files found")

print(f"\n📊 Regional Analysis Capability: {'Enabled' if regional_analysis_possible else 'Limited - using demonstration framework'}")

# =============================================================================
# REGIONAL WATER BALANCE ANALYSIS
# =============================================================================

print(f"\n💧 Regional Water Balance Analysis...")

if regional_analysis_possible and regional_summa_data is not None:
    try:
        print(f"✅ Analyzing regional water balance components")
        
        # Extract key water balance variables
        available_vars = list(regional_summa_data.data_vars.keys())
        water_balance_vars = {
            'Total Soil Water': 'scalarTotalSoilWat',
            'Snow Water Equivalent': 'scalarSWE', 
            'Surface Runoff': 'scalarSurfaceRunoff',
            'Evapotranspiration': 'scalarLatHeatTotal',
            'Net Precipitation': 'scalarNetPrecipitation'
        }
        
        # Analyze available water balance components
        print(f"\n💧 Water Balance Components Available:")
        regional_water_components = []
        for var_name, var_key in water_balance_vars.items():
            if var_key in available_vars:
                var_data = regional_summa_data[var_key]
                if 'hru' in var_data.dims:
                    mean_value = var_data.mean().values
                    regional_water_components.append((var_name, mean_value, var_key))
                    print(f"   💧 {var_name}: Regional mean = {mean_value:.2f}")
                else:
                    print(f"   📋 {var_name}: Available but requires processing")
            else:
                print(f"   ❌ {var_name}: Not available in outputs")
        
        # Calculate regional water balance if sufficient data
        if len(regional_water_components) >= 2:
            print(f"\n🌊 Regional Water Balance Summary:")
            print(f"   Analysis period: {regional_summa_data.time.min().values} to {regional_summa_data.time.max().values}")
            print(f"   Spatial coverage: {regional_summa_data.dims.get('hru', 'N/A')} computational units")
            print(f"   Water balance components: {len(regional_water_components)} variables analyzed")
        
    except Exception as e:
        print(f"   ⚠️  Water balance analysis failed: {e}")
        regional_analysis_possible = False


# =============================================================================
# INTER-WATERSHED COMPARATIVE ANALYSIS
# =============================================================================

print(f"\n🔄 Inter-Watershed Comparative Analysis...")

# Load watershed boundaries for comparative analysis
basins_gdf['elevation'] = hru_gdf['elev_mean']
if 'basins_gdf' in locals() and basins_gdf is not None:
    watershed_count = len(basins_gdf)
    print(f"✅ Watershed boundaries available for comparative analysis")
    print(f"   Watersheds: {watershed_count} independent drainage systems")
    
    if regional_analysis_possible and regional_summa_data is not None:
        print(f"\n🏔️  Watershed Characteristics Analysis:")
        
        # Analyze watershed diversity if elevation data available
        if 'elevation' in basins_gdf.columns:
            elev_stats = {
                'min': basins_gdf['elevation'].min(),
                'max': basins_gdf['elevation'].max(),
                'mean': basins_gdf['elevation'].mean(),
                'std': basins_gdf['elevation'].std()
            }
            print(f"   Elevation diversity: {elev_stats['min']:.0f}m to {elev_stats['max']:.0f}m")
            print(f"   Mean elevation: {elev_stats['mean']:.0f}m ± {elev_stats['std']:.0f}m")
            
            # Categorize watersheds by elevation
            elevation_categories = []
            for idx, row in basins_gdf.iterrows():
                if row['elevation'] < 200:
                    elevation_categories.append('Coastal')
                elif row['elevation'] < 500:
                    elevation_categories.append('Lowland')
                elif row['elevation'] < 1000:
                    elevation_categories.append('Highland')
                else:
                    elevation_categories.append('Alpine')
            
            basins_gdf['elevation_category'] = elevation_categories
            category_counts = pd.Series(elevation_categories).value_counts()
            
            print(f"   Watershed types by elevation:")
            for category, count in category_counts.items():
                print(f"     {category}: {count} watersheds")
        
        # Analyze area distribution
        if hasattr(basins_gdf, 'geometry'):
            area_km2 = basins_gdf.geometry.area / 1e6
            print(f"   Area distribution: {area_km2.min():.1f} to {area_km2.max():.1f} km²")
            print(f"   Mean watershed size: {area_km2.mean():.1f} km²")

else:
    print(f"⚠️  Watershed boundary data not available for comparative analysis")


# =============================================================================
# REGIONAL STREAMFLOW OUTLET ANALYSIS
# =============================================================================

if regional_routing_data is not None and 'IRFroutedRunoff' in regional_routing_data.data_vars:
    try:
        streamflow_data = regional_routing_data['IRFroutedRunoff']
        n_segments = streamflow_data.sizes.get('seg', 0)
        
        print(f"✅ Regional streamflow analysis available")
        print(f"   Stream segments: {n_segments}")
        print(f"   Multiple outlets: Coastal discharge points around Iceland")
        
        # Analyze streamflow patterns across outlets
        if n_segments > 0:
            # Calculate basic streamflow statistics
            mean_flows = streamflow_data.mean(dim='time')
            max_flows = streamflow_data.max(dim='time')
            
            print(f"\n🌊 Regional Streamflow Patterns:")
            print(f"   Mean flow range: {mean_flows.min().values:.2f} to {mean_flows.max().values:.2f} m³/s")
            print(f"   Peak flow range: {max_flows.min().values:.2f} to {max_flows.max().values:.2f} m³/s")
            print(f"   Flow variability: Multiple independent coastal discharge systems")
            
            # Identify major outlets (demonstration)
            if n_segments >= 5:
                largest_outlets = mean_flows.argsort()[-5:]  # Top 5 outlets by mean flow
                print(f"   Major outlets: {len(largest_outlets)} primary discharge systems identified")
        
    except Exception as e:
        print(f"   ⚠️  Streamflow analysis failed: {e}")


if 'basins_gdf' in locals() and basins_gdf is not None and not basins_gdf.empty:
    
    fig, axes = plt.subplots(2, 3, figsize=(20, 14))
    
    # Regional watersheds overview (top left)
    ax1 = axes[0, 0]
    if 'GRU_ID' in basins_gdf.columns:
        basins_gdf.plot(ax=ax1, column='GRU_ID', cmap='tab20', 
                       edgecolor='black', linewidth=0.5, alpha=0.8, legend=False)
    else:
        basins_gdf.plot(ax=ax1, cmap='tab20', 
                       edgecolor='black', linewidth=0.5, alpha=0.8, legend=False)
    
    ax1.set_title(f'Regional Watershed Systems\n{len(basins_gdf)} Independent Drainages', 
                 fontweight='bold')
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.grid(True, alpha=0.3)
    
    # Watershed elevation distribution (top middle)
    ax2 = axes[0, 1]
    if 'elevation' in basins_gdf.columns:
        basins_gdf.plot(ax=ax2, column='elevation', cmap='terrain', 
                       edgecolor='gray', linewidth=0.3, alpha=0.8,
                       legend=True, legend_kwds={'label': 'Elevation (m)', 'shrink': 0.6})
        ax2.set_title('Elevation Gradient Analysis', fontweight='bold')
    else:
        ax2.text(0.5, 0.5, 'Elevation\nanalysis\nrequires data', 
                transform=ax2.transAxes, ha='center', va='center',
                fontsize=12, bbox=dict(facecolor='lightgray', alpha=0.5))
        ax2.set_title('Elevation Analysis', fontweight='bold')
    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Latitude')
    ax2.grid(True, alpha=0.3)
    
    # Watershed size distribution (top right)
    ax3 = axes[0, 2]
    if hasattr(basins_gdf, 'geometry'):
        watershed_areas = basins_gdf.geometry.area / 1e6  # Convert to km²
        ax3.hist(watershed_areas, bins=min(15, len(basins_gdf)//2), 
                color='lightblue', alpha=0.7, edgecolor='navy')
        ax3.set_xlabel('Watershed Area (km²)')
        ax3.set_ylabel('Number of Watersheds')
        ax3.set_title('Watershed Size Distribution', fontweight='bold')
        ax3.grid(True, alpha=0.3)
        
        # Add statistics
        stats_text = (f"Count: {len(watershed_areas)}\n"
                     f"Mean: {watershed_areas.mean():.0f} km²\n"
                     f"Range: {watershed_areas.min():.0f}-{watershed_areas.max():.0f} km²")
        ax3.text(0.98, 0.98, stats_text, transform=ax3.transAxes,
                bbox=dict(facecolor='white', alpha=0.8), fontsize=9,
                ha='right', va='top')
    
    # Watershed type analysis (bottom left)
    ax4 = axes[1, 0]
    if 'elevation_category' in basins_gdf.columns:
        category_counts = basins_gdf['elevation_category'].value_counts()
        colors = ['lightcoral', 'lightgreen', 'lightblue', 'lightyellow']
        bars = ax4.bar(category_counts.index, category_counts.values, 
                      color=colors[:len(category_counts)], alpha=0.7, edgecolor='black')
        ax4.set_ylabel('Number of Watersheds')
        ax4.set_title('Watershed Types by Elevation', fontweight='bold')
        ax4.grid(True, alpha=0.3, axis='y')
        
        # Add value labels
        for bar, count in zip(bars, category_counts.values):
            ax4.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.1,
                    f'{count}', ha='center', va='bottom', fontsize=10)
    else:
        ax4.text(0.5, 0.5, 'Watershed\ntype analysis\nrequires\nelevation data', 
                transform=ax4.transAxes, ha='center', va='center',
                fontsize=11, bbox=dict(facecolor='lightgray', alpha=0.5))
        ax4.set_title('Watershed Type Analysis', fontweight='bold')
    
    # Regional water resources summary (bottom middle)
    ax5 = axes[1, 1]
    
    # Create summary metrics visualization
    if hasattr(basins_gdf, 'geometry'):
        total_area = basins_gdf.geometry.area.sum() / 1e6
        metrics = {
            'Watersheds': len(basins_gdf),
            'Area (1000 km²)': total_area / 1000,
            'Mean Size (km²)': watershed_areas.mean(),
            'Outlets': len(basins_gdf)  # Each watershed has coastal outlet
        }
        
        y_pos = np.arange(len(metrics))
        values = list(metrics.values())
        
        bars = ax5.barh(y_pos, values, color='steelblue', alpha=0.7)
        ax5.set_yticks(y_pos)
        ax5.set_yticklabels(list(metrics.keys()))
        ax5.set_xlabel('Value')
        ax5.set_title('Regional Summary Metrics', fontweight='bold')
        ax5.grid(True, alpha=0.3, axis='x')
        
        # Add value labels
        for i, (bar, value) in enumerate(zip(bars, values)):
            ax5.text(bar.get_width() + max(values)*0.01, bar.get_y() + bar.get_height()/2.,
                    f'{value:.0f}', ha='left', va='center', fontsize=9)
    
    # Tutorial series progression (bottom right)
    ax6 = axes[1, 2]
    
    # Show modeling evolution
    tutorial_progression = {
        'Scale': ['Basin\n(02a)', 'Sub-basin\n(02b)', 'Elevation\n(02c)', 'Regional\n(03a)'],
        'Units': [1, 15, 45, len(basins_gdf)],  # Approximate values
        'Complexity': ['Low', 'Medium', 'High', 'Regional']
    }
    
    x_pos = np.arange(len(tutorial_progression['Scale']))
    bars = ax6.bar(x_pos, tutorial_progression['Units'], 
                   color=['lightcoral', 'lightgreen', 'lightblue', 'gold'], 
                   alpha=0.7, edgecolor='navy')
    
    ax6.set_xlabel('Tutorial Progression')
    ax6.set_ylabel('Computational Units')
    ax6.set_title('Modeling Scale Evolution', fontweight='bold')
    ax6.set_xticks(x_pos)
    ax6.set_xticklabels(tutorial_progression['Scale'])
    ax6.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for i, (bar, units, complexity) in enumerate(zip(bars, tutorial_progression['Units'], tutorial_progression['Complexity'])):
        ax6.text(bar.get_x() + bar.get_width()/2., bar.get_height() + max(tutorial_progression['Units'])*0.02,
                f'{units}\n({complexity})', ha='center', va='bottom', fontsize=9)
    
    plt.suptitle(f'Regional Multi-Watershed Analysis - Iceland Domain Assessment', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print(f"✅ Regional visualization complete")

else:
    print(f"❌ Cannot create regional visualization - watershed data not available")


## Summary: Regional Domain Modeling
This tutorial successfully demonstrated the transition from watershed-scale to regional-scale hydrological modeling by implementing comprehensive regional domain analysis using Iceland as an exemplary case study. Through the Iceland regional domain, we illustrated how the same standardized CONFLUENCE workflow framework seamlessly scales from single-watershed applications to multi-watershed regional systems while capturing the complete hydrological picture of entire geographic regions, representing a paradigm shift from watershed-centric to region-centric hydrological analysis.

## Key Methodological Achievements
The tutorial established regional multi-watershed modeling capabilities through systematic identification and delineation of all independent drainage systems within geographic boundaries rather than topographic watershed limits. Coastal watershed integration was demonstrated through explicit inclusion of ocean-draining basins that traditional watershed modeling approaches often overlook, ensuring complete regional water accounting. Bounding box delineation methodology was successfully implemented to define regions using administrative or geographic boundaries rather than pour point specifications, enabling comprehensive coverage of entire administrative units or natural regions.

## Scientific Process Understanding
The evaluation demonstrated CONFLUENCE's capability to represent multiple independent drainage systems within a unified modeling framework, enabling simultaneous analysis of diverse hydrological regimes without artificial connectivity assumptions. Regional water resources assessment was achieved through integrated modeling of coastal watersheds, highland systems, and diverse topographic settings that capture the complete spectrum of hydrological conditions within national boundaries. Multi-outlet discharge analysis was established through systematic handling of numerous independent coastal discharge points, providing comprehensive understanding of regional water export patterns.

## Framework Scalability Validation
This tutorial confirmed CONFLUENCE's seamless regional scaling by successfully applying identical workflow principles from watershed-specific applications to comprehensive regional domains without requiring fundamental architectural modifications. The model-agnostic preprocessing approach proved equally effective for regional-scale spatial processing and multi-watershed system configuration, demonstrating robust design principles across dramatically different spatial paradigms. Computational efficiency optimization was maintained through intelligent regional processing strategies that balance comprehensive spatial coverage with practical computational requirements for regional water management applications.

This advancement from watershed to regional modeling establishes essential capabilities for national water resources assessment, climate impact evaluation across administrative boundaries, and regional policy applications that require understanding of hydrological processes across entire geographic regions rather than individual watershed systems.

### Next Focus: Continental Scale Domain Modelling 

**Ready to explore continential domain scale simulations?** → **[Tutorial 03b: Continental Domain Scale - North America](./03b_domain_continental.ipynb)**